# "Eleições RJ: Votação dos Deputados por Bairros (capital)"
> "dados do TSE"

- toc: true
- branch: master
- badges: true
- comments: false
- numbersections: true
- categories: [eleições, RJ, bairros]
- image: images/some_folder/your_image.png
- hide: false
- search_exclude: true

In [1]:
#hide

import os
from IPython.core.display import display, HTML
from IPython.display import IFrame

import pandas as pd

import electionsBR as el
import LabFunctions as lab
from EppDataBase import EppDataBase

def get_dict_candidatxs(uf,ano,cargo,partidos,only_elected=True):
    elected = get_elected_candidates_dict(uf,partidos,cargos=[cargo],anos=[ano],only_elected=only_elected)
    df = elected[cargo][ano].copy()
    # else:
    #     pass
    # df['num_nome'] = df['NUMERO_CANDIDATO'].astype(str) + ' - ' + df['NOME_CANDIDATO']
    df['num_nome'] = df['NUMERO_CANDIDATO'] + ' - ' + df['NOME_CANDIDATO']
    return df.set_index('NUMERO_CANDIDATO')['num_nome'].to_dict()

def show_folium(m):
    data = m.get_root().render()
    data_fixed_height = data.replace('width: 100%;height: 100%', 'width: 100%').replace('height: 100.0%;', 'height: 609px;', 1)
    # display(HTML(data_fixed_height))
    return data_fixed_height

def plot_candidates_bars(epp,ano,uf,n_turno,cargo,municipio,partidos,only_elected,group_level,selected_col,relative, nr_partidos,nrows=None):
    selection_dict = {'NR_TURNO':n_turno,'DS_CARGO':cargo,'NM_MUNICIPIO':municipio}
    # dict_candidatxs = get_dict_candidatxs(elected,ano,cargo)
    # partidos=partidos_esquerda

    if cargo == 'VEREADOR':
        # dict_candidatxs = {}
        
        df = epp.get_results_df(ano,uf)
        df_dict = df.loc[df['nr_partido'].isin(nr_partidos)].copy()
        df_dict['NR_VOTAVEL'] = df_dict['NR_VOTAVEL'].astype(str)
        df_dict['nome'] = df_dict['NR_VOTAVEL'] + ' - ' + df_dict['NM_VOTAVEL']
        dict_candidatxs = df_dict.set_index('NR_VOTAVEL')['nome'].to_dict()
        only_elected = False
    else:
        dict_candidatxs = get_dict_candidatxs(uf,ano,cargo,partidos=partidos_esquerda,only_elected=only_elected)
        dict_candidatxs['13'] = '13 - PT'
        dict_candidatxs['50'] = '50 - PSOL'
        dict_candidatxs['65'] = '65 - PCdoB'

    df_bairros = lab.get_group_results(ano,uf,selection_dict,group_level,selected_col=selected_col,relative=relative,dict_candidatxs=dict_candidatxs)

    # selected_cols = [col for col in df_bairros.columns if col[:2] in numeros_partidos]
    # df_bairros = df_bairros[selected_cols]


    sufix = 'eleitos_' if only_elected else 'todos_'
    sufix += 'relativo' if relative else 'absoluto'
    folder_path = 'results/votacao_NM_BAIRRO'
    os.makedirs(folder_path,exist_ok=True)
    fname = f'{ano}_RJ_{cargo}_{"_".join(partidos)}_{sufix}.csv'.replace(' ','_')
    df_bairros = df_bairros.reset_index(drop=True)
    if not relative:
        df_bairros.iloc[:,2:] = df_bairros.iloc[:,2:].astype(int)
        # df_bairros.iloc[:,2:] = df_bairros.select_dtypes(include='number').astype(int)
    df_bairros = df_bairros.copy()
    df_bairros['SOMA'] = df_bairros.sum(axis=1,numeric_only=True)
    df_bairros = df_bairros.sort_values(by='SOMA',ascending=False)
    df_bairros.to_csv(f'{folder_path}/{fname}',index=False,sep=';')
    if nrows:
        df_bairros = df_bairros.head(nrows)
    df_bairros.index = range(1,len(df_bairros)+1)

    # hover_color = '#00BBBB'
    # styles = [dict(selector="tr:hover", props=[("background-color", f"{hover_color}")])]
    styles = [dict(selector="tr:hover", props=[("font-weight", "bold"),('font-size','12px',)])]
    styles.append(dict(selector="", props=[("font-size", "8px")]))
    styles.append(dict(selector="", props=[("line-height", "6px")]))
    styles.append(dict(selector="", props=[("text-align", "center")]))
    
    df_style = df_bairros.style
    df_style = df_style.set_properties(**{
                'padding': '0px',
                'font-size':'8px',
                'line-height':'10px'})

    # df_style= df_style.set_table_styles(styles)
    df_style = df_style.background_gradient() #.bar(color='#d65f5f')
    df_style= df_style.set_table_styles(styles)



    return df_style


def get_elected_candidates_dict(uf,partidos,cargos,anos,only_elected=True,verbose=False):
    if only_elected:
        folder_path = 'results/eleitos'
    else:
        folder_path = 'results/candidatos'
    os.makedirs(folder_path,exist_ok=True)
    elected = {}
    for cargo in cargos:
        cargo_api = ' '.join([x.capitalize() for x in cargo.split()])
        elected[cargo] = {}
        for ano in anos:
            fname = f'{ano}_{uf}_{cargo}_{"_".join(partidos)}.csv'.replace(' ','_')
            file_path = f'{folder_path}/{fname}'
            if verbose:
                print(file_path)
            if not os.path.exists(file_path):
                if only_elected:
                    cand = el.get_candidates(year=ano, position=cargo_api,only_elected=only_elected,
                        columns=["NUMERO_CANDIDATO", "NOME_CANDIDATO", "SIGLA_UF", "NUM_TURNO", "SIGLA_PARTIDO", "COMPOSICAO_LEGENDA"])
                else:
                    cand = el.get_candidates(year=ano, position=cargo_api,
                        columns=["NUMERO_CANDIDATO", "NOME_CANDIDATO", "SIGLA_UF", "NUM_TURNO", "SIGLA_PARTIDO", "COMPOSICAO_LEGENDA"])

                cand = cand.query(f"SIGLA_UF == '{uf}' ").query(f"SIGLA_PARTIDO in {partidos}")
                cand.to_csv(file_path,index=False)
            else:
                cand = pd.read_csv(file_path)
                # cand['']
                cand['NUMERO_CANDIDATO'] = cand['NUMERO_CANDIDATO'].astype(str)
            
            elected[cargo][ano] = cand
            
    return elected

def plot_folium_map(df_bairros,bairros_geo='',nome_cand='',filename='test.html'):
    if not bairros_geo:
        bairros_geo = 'geojson/limites_bairros_RJ_edit.geojson'
    
    map = lab.get_map(bairros_geo,df_bairros,nome_cand,0.0)

    folder_path = '../maps/html'
    os.makedirs(folder_path,exist_ok=True)
    file_path = f'{folder_path}/{filename}'
    map.save(file_path)
    display(IFrame(f'/covid/_notebooks/{file_path}',1000,400))




In [2]:
#hide

# definições gerais

n_turno = 1
uf = 'RJ'
anos = [2014,2018]
partidos_esquerda = ['PT','PSOL','PC do B']
cand_elections_fed = {}
epp = EppDataBase()

cargos= ["DEPUTADO FEDERAL","DEPUTADO ESTADUAL"]

## Deputado Federal - 2018

**Tabela de votação por bairro - absoluto**

In [3]:
#hide_input
#collapse_output

ano = 2018
cargo = 'DEPUTADO FEDERAL' 
dict_candidatxs = get_dict_candidatxs(uf,ano,cargo,partidos=partidos_esquerda)
lista_cand = list(dict_candidatxs.keys())

municipio= 'RIO DE JANEIRO'
print(f'município selecionado: {municipio}')
group_level = 'NM_BAIRRO' #  NM_BAIRRO NM_LOCAL_VOTACAO
selected_col='NR_VOTAVEL'
nr_partidos=[13,50,65]
relative = False
only_elected=True
nrows=50

plot_candidates_bars(epp,ano,uf,n_turno,cargo,municipio,partidos_esquerda,only_elected,group_level,selected_col,relative, nr_partidos,nrows)

município selecionado: RIO DE JANEIRO


,NM_MUNICIPIO,NM_BAIRRO,1377 - BENEDITA SOUZA DA SILVA SAMPAIO,5005 - JEAN WYLLYS DE MATOS SANTOS,5050 - MARCELO RIBEIRO FREIXO,5077 - TALIRIA PETRONE SOARES,5080 - GLAUBER DE MEDEIROS BRAGA,6565 - JANDIRA FEGHALI,13 - PT,50 - PSOL,65 - PCdoB,SOMA
1,RIO DE JANEIRO,TIJUCA,486,945,11770,3921,893,1398,769,556,29,20767
2,RIO DE JANEIRO,CAMPO GRANDE,774,364,10313,1684,762,1593,1374,462,61,17387
3,RIO DE JANEIRO,COPACABANA,597,1174,7799,2468,545,1222,626,332,20,14783
4,RIO DE JANEIRO,BOTAFOGO,204,658,5018,1895,415,686,289,240,7,9412
5,RIO DE JANEIRO,REALENGO,513,263,4823,1139,252,819,735,249,29,8822
6,RIO DE JANEIRO,TAQUARA,323,214,4981,1209,283,647,446,224,18,8345
7,RIO DE JANEIRO,VILA ISABEL,214,334,4449,1368,301,609,305,204,13,7797
8,RIO DE JANEIRO,BANGU,407,215,4055,866,240,751,708,187,39,7468
9,RIO DE JANEIRO,LARANJEIRAS,132,593,3720,1640,305,577,158,151,15,7291
10,RIO DE JANEIRO,SANTA CRUZ,622,146,3607,577,191,834,898,158,21,7054


**Tabela de votação por bairro - relativo**

In [4]:
#hide_input
#collapse_output
 
relative = True
plot_candidates_bars(epp,ano,uf,n_turno,cargo,municipio,partidos_esquerda,only_elected,group_level,selected_col,relative, nr_partidos,nrows)

,NM_MUNICIPIO,NM_BAIRRO,1377 - BENEDITA SOUZA DA SILVA SAMPAIO,5005 - JEAN WYLLYS DE MATOS SANTOS,5050 - MARCELO RIBEIRO FREIXO,5077 - TALIRIA PETRONE SOARES,5080 - GLAUBER DE MEDEIROS BRAGA,6565 - JANDIRA FEGHALI,13 - PT,50 - PSOL,65 - PCdoB,SOMA
1,RIO DE JANEIRO,GLÓRIA,0.006161,0.017687,0.118641,0.048490,0.009340,0.019873,0.007949,0.005167,0.000000,0.233307
2,RIO DE JANEIRO,LARANJEIRAS,0.004170,0.018732,0.117510,0.051805,0.009635,0.018227,0.004991,0.004770,0.000474,0.230312
3,RIO DE JANEIRO,HUMAITA,0.004611,0.014961,0.109950,0.053182,0.008505,0.014551,0.003689,0.005226,0.000102,0.214776
4,RIO DE JANEIRO,SANTA TERESA,0.013056,0.012597,0.103464,0.040218,0.008267,0.016730,0.011809,0.005117,0.000394,0.211652
5,RIO DE JANEIRO,BAIRRO DE FÁTIMA,0.007593,0.011863,0.113097,0.041126,0.006011,0.014078,0.009174,0.006169,0.000633,0.209744
6,RIO DE JANEIRO,MARACANÃ,0.004047,0.009727,0.116365,0.045509,0.010792,0.012922,0.004899,0.005183,0.000142,0.209585
7,RIO DE JANEIRO,COSME VELHO,0.004828,0.013412,0.108548,0.050608,0.007332,0.011981,0.008226,0.003755,0.000000,0.208691
8,RIO DE JANEIRO,LAPA,0.006944,0.016204,0.112037,0.031944,0.006019,0.018519,0.007870,0.006944,0.000463,0.206944
9,RIO DE JANEIRO,CATETE,0.010153,0.011256,0.104617,0.035711,0.007416,0.017348,0.008166,0.004856,0.000177,0.199700
10,RIO DE JANEIRO,FLAMENGO,0.005566,0.013795,0.100718,0.037714,0.010891,0.015771,0.005203,0.004397,0.000000,0.194055


**Mapa por Bairro**

In [5]:
#hide_input
#collapse_output

i = 1
num = lista_cand[i]
nome_cand = dict_candidatxs[num]

group_level = 'NM_BAIRRO' #  NM_BAIRRO NM_LOCAL_VOTACAO
selected_col='NR_VOTAVEL'
relative = True
# relative = False
use_max_scale = False
selection_dict = {'NR_TURNO':n_turno,'NM_MUNICIPIO':municipio,'DS_CARGO':cargo}
df_bairros = lab.get_group_results(ano,uf,selection_dict,group_level,selected_col=selected_col,relative=relative,dict_candidatxs=dict_candidatxs)
filename = f'{ano}_{uf}_{municipio}_{cargo}_{nome_cand}.html'.replace(' ','_')
bairros_geo = 'geojson/limites_bairros_RJ_edit.geojson'

print(f'Arquivo usado: {filename}')
plot_folium_map(df_bairros,bairros_geo,nome_cand,filename)


Arquivo usado: 2018_RJ_RIO_DE_JANEIRO_DEPUTADO_FEDERAL_5005_-_JEAN_WYLLYS_DE_MATOS_SANTOS.html


## Deputado Estadual - 2018
**Tabela de votação por bairro - absoluto**

In [6]:
#hide_input
#collapse_output

ano - 2018
cargo = 'DEPUTADO ESTADUAL'

municipio= 'RIO DE JANEIRO'
print(f'município selecionado: {municipio}')
group_level = 'NM_BAIRRO' #  NM_BAIRRO NM_LOCAL_VOTACAO
selected_col='NR_VOTAVEL'
nr_partidos=[13,50,65]
relative = False
only_elected=True

plot_candidates_bars(epp,ano,uf,n_turno,cargo,municipio,partidos_esquerda,only_elected,group_level,selected_col,relative, nr_partidos,nrows)

município selecionado: RIO DE JANEIRO


,NM_MUNICIPIO,NM_BAIRRO,13000 - WALDECK CARNEIRO DA SILVA,13103 - ROSANGELA DE OLIVEIRA ZEIDAN,13567 - ANDRÉ LUIZ CECILIANO,50000 - ELIOMAR DE SOUZA COELHO,50007 - RENATA DA SILVA SOUZA,50050 - DANIELLA MONTEIRO DA SILVA,50123 - FLAVIO ALVES SERAFINI,50888 - MONICA SANTOS FRANCISCO,65656 - REJANE DE ALMEIDA,13 - PT,50 - PSOL,65 - PCdoB,SOMA
1,RIO DE JANEIRO,TIJUCA,213,105,177,2274,3243,1148,1477,2821,509,1023,1631,51,14672
2,RIO DE JANEIRO,COPACABANA,192,99,143,1924,2404,816,902,1627,275,846,1116,52,10396
3,RIO DE JANEIRO,CAMPO GRANDE,157,939,168,667,942,450,563,646,981,1510,1089,107,8219
4,RIO DE JANEIRO,BOTAFOGO,113,40,52,1384,1844,602,601,1163,160,508,719,20,7206
5,RIO DE JANEIRO,LARANJEIRAS,92,34,38,1157,1505,461,476,1117,95,242,485,18,5720
6,RIO DE JANEIRO,TAQUARA,39,232,126,612,894,461,417,598,488,551,631,26,5075
7,RIO DE JANEIRO,VILA ISABEL,126,42,58,789,1016,435,575,727,260,390,614,34,5066
8,RIO DE JANEIRO,REALENGO,276,99,103,385,639,442,334,532,645,829,637,36,4957
9,RIO DE JANEIRO,BARRA DA TIJUCA,62,31,110,676,1283,497,367,814,102,303,499,5,4749
10,RIO DE JANEIRO,BANGU,110,219,657,352,538,305,290,398,396,818,541,40,4664


**Tabela de votação por bairro - relativo**

In [7]:
#hide_input
#collapse_output
 
relative = True
plot_candidates_bars(epp,ano,uf,n_turno,cargo,municipio,partidos_esquerda,only_elected,group_level,selected_col,relative, nr_partidos, nrows)

,NM_MUNICIPIO,NM_BAIRRO,13000 - WALDECK CARNEIRO DA SILVA,13103 - ROSANGELA DE OLIVEIRA ZEIDAN,13567 - ANDRÉ LUIZ CECILIANO,50000 - ELIOMAR DE SOUZA COELHO,50007 - RENATA DA SILVA SOUZA,50050 - DANIELLA MONTEIRO DA SILVA,50123 - FLAVIO ALVES SERAFINI,50888 - MONICA SANTOS FRANCISCO,65656 - REJANE DE ALMEIDA,13 - PT,50 - PSOL,65 - PCdoB,SOMA
1,RIO DE JANEIRO,COSME VELHO,0.001967,0.001073,0.001073,0.027718,0.040057,0.010908,0.014306,0.053469,0.004292,0.013948,0.015200,0.000536,0.184549
2,RIO DE JANEIRO,HUMAITA,0.001947,0.001844,0.000820,0.029511,0.054309,0.016190,0.013526,0.041398,0.003177,0.006558,0.012501,0.000205,0.181986
3,RIO DE JANEIRO,LARANJEIRAS,0.002906,0.001074,0.001200,0.036548,0.047541,0.014562,0.015036,0.035284,0.003001,0.007644,0.015320,0.000569,0.180687
4,RIO DE JANEIRO,GLÓRIA,0.002385,0.002981,0.002186,0.024444,0.038752,0.012122,0.012520,0.026232,0.005167,0.010334,0.016097,0.000000,0.153219
5,RIO DE JANEIRO,BOTAFOGO,0.002323,0.000822,0.001069,0.028455,0.037912,0.012377,0.012356,0.023911,0.003290,0.010444,0.014782,0.000411,0.148153
6,RIO DE JANEIRO,FLAMENGO,0.002299,0.001170,0.001452,0.027388,0.036463,0.011092,0.012060,0.025734,0.003711,0.008228,0.014884,0.000323,0.144805
7,RIO DE JANEIRO,SANTA TERESA,0.003412,0.004986,0.000984,0.021323,0.036085,0.009513,0.008529,0.023488,0.002887,0.015287,0.015155,0.000722,0.142370
8,RIO DE JANEIRO,MARACANÃ,0.002059,0.000852,0.001065,0.022151,0.030955,0.014341,0.016826,0.021583,0.005183,0.009088,0.017181,0.000497,0.141782
9,RIO DE JANEIRO,LAPA,0.001389,0.001852,0.000926,0.016204,0.042593,0.009259,0.013426,0.016667,0.005093,0.012963,0.018056,0.000463,0.138889
10,RIO DE JANEIRO,JARDIM BOTÂNICO,0.004270,0.001128,0.001370,0.021832,0.037541,0.010231,0.007009,0.029566,0.001289,0.009990,0.011440,0.000322,0.135986


## Vereador - 2020

In [9]:
#hide_input
#collapse_output

ano = 2020
cargo = 'VEREADOR'

municipio= 'RIO DE JANEIRO'
print(f'município selecionado: {municipio}')
group_level = 'NM_BAIRRO' #  NM_BAIRRO NM_LOCAL_VOTACAO
selected_col='NR_VOTAVEL'
nr_partidos=[13,50,65]
relative = False
only_elected=True

df_style = plot_candidates_bars(epp,ano,uf,n_turno,cargo,municipio,partidos_esquerda,only_elected,group_level,selected_col,relative, nr_partidos,nrows)

df_style.background_gradient()

município selecionado: RIO DE JANEIRO
Adicionando resultado...
Lendo dados do arquivo 'data/votacao_secao_2020_RJ/votacao_secao_2020_RJ.csv'.


**Tabela de votação por bairro - relativo**

In [10]:
#hide_input
#collapse_output

relative = True
plot_candidates_bars(epp,ano,uf,n_turno,cargo,municipio,partidos_esquerda,only_elected,group_level,selected_col,relative, nr_partidos, nrows)